In [1]:
%run util_phys.py
import ROOT as R

histnameschanged = ['S', 'S', 'S', 'S', '', '', '']
runs = ['31365_p1v1nt', '31365_p1v1bos', '31365_p2v1bos',
        '38121_p2v1bos', '38121_recook', '38121_p1v1bos', '38121_p2v1ntp']
beam_energy = [E16_E0, E16_E0, E16_E0, E1F_E0, E1F_E0, E1F_E0, E1F_E0]
colors = [R.kBlue+2, R.kGreen+2, R.kRed+2, R.kYellow+2, R.kOrange+2, R.kBlack+2, R.kRed]

def add_sectors(run, hn_tmpl='eid/heid_s%d', hn_tmpl_out='h%s'):
    dir_data = '/home/ephelps/projects/phys-ana-omega/e1f6/test'
    fin = R.TFile('%s/%s_eskim.root'%(dir_data, run))
    h = fin.Get(hn_tmpl%1).Clone(hn_tmpl_out%run)
    for sect in range (2,7):
        h.Add(fin.Get(hn_tmpl%sect))
    h.SetDirectory(0)
    fin.Close()
    return h

def get_fcq(run):
    dir_data = '/home/ephelps/projects/phys-ana-omega/e1f6/test'
    fin = R.TFile('%s/%s_eskim.root'%(dir_data, run))
    lum = fin.Get('lumblocks/lumblocks')
    fcq, fcq_trip = 0, 0
    for lb in lum:
        fcq_trip += lb.fc_charge_ltcorr if lb.fc_charge_ltcorr < 0.5 else 0
        fcq += lb.fc_charge_ltcorr
    return (fcq, fcq_trip)

In [2]:
c = R.TCanvas("ceid","EID cuts",800,600)
c.Divide(2,2)

In [3]:
# eid cuts, absolute
hs_e_ind = [add_sectors(run, 'eid/heidind_s%d', 'h%s_ind') for run in runs]
hs_e_cum = [add_sectors(run, 'eid/heid_s%d', 'h%s_cum') for run in runs]

hstack_e_ind = R.THStack('hstack_e_ind', 'eid cuts, independent')
hstack_e_cum = R.THStack('hstack_e_cum', 'eid cuts, cumulative')

for i, (run, h, hn, e0) in enumerate(zip(runs, hs_e_ind, hs_e_cum, beam_energy)):
    h.SetTitle('eid cuts, independent, %s'%run)
    hn.SetTitle('eid cuts, cumulative, %s'%run)
    h.SetLineColor(colors[i])
    hn.SetLineColor(colors[i])
    hstack_e_ind.Add(h)
    hstack_e_cum.Add(hn)

p1 = c.cd(1)
hstack_e_ind.Draw('nostack')
p1.SetGrid()
p1.BuildLegend()

p2 = c.cd(2)
hstack_e_cum.Draw('nostack')
p2.SetGrid()
p2.BuildLegend()

<ROOT.TLegend object ("TPave") at 0x6b7b1a0>

In [4]:
# eid cuts, normalized sample
hs_e_ind_norm = [add_sectors(run, 'eid/heidind%s_sample_2_2_s%%d'%xtrachar, 'h%s_ind') for (run,xtrachar) in zip(runs,histnameschanged)]
hs_e_cum_norm = [add_sectors(run, 'eid/heid_sample_2_2_s%d', 'h%s_cum_norm') for run in runs]

hstack_e_ind_norm = R.THStack('hstack_e_ind_norm', 'eid cuts, independent, normalized')
hstack_e_cum_norm = R.THStack('hstack_e_cum_norm', 'eid cuts, cumulative, normalized')

for i, (run, h, hn, e0) in enumerate(zip(runs, hs_e_ind_norm, hs_e_cum_norm, beam_energy)):
    fcq = get_fcq('38121_p2v1bos')[1] if run in ['38121_recook', '38121_p2v1ntp'] else get_fcq(run)[1]
    fcq = get_fcq('31365_p1v1bos')[1] if run == '31365_p1v1nt' else fcq
    h.Scale(fcq/vgflux(2, 2, e0))
    hn.Scale(fcq/vgflux(2, 2, e0))
    h.SetTitle('eid cuts, independent, normalized, %s'%run)
    hn.SetTitle('eid cuts, cumulative, normalized, %s'%run)
    h.SetLineColor(colors[i])
    hn.SetLineColor(colors[i])
    hstack_e_ind_norm.Add(h)
    hstack_e_cum_norm.Add(hn)

p3 = c.cd(3)
hstack_e_ind_norm.Draw('nostack')
p3.SetGrid()
p3.BuildLegend()

p4 = c.cd(4)
hstack_e_cum_norm.Draw('nostack')
p4.SetGrid()
p4.BuildLegend()

<ROOT.TLegend object ("TPave") at 0x6c34520>

In [5]:
c.Draw()

In [ ]:
run = '38121_p2v1bos'
c2 = R.TCanvas('c2', 'c2')
h = add_sectors(run, 'eid/heidindS_sample_2_2_s%d', 'h%s_indsamp')
h.Draw()
h.Scale(get_fcq('38121_p2v1bos')[1]/vgflux(2, 2, E1F_E0))
c2.Draw()

In [ ]:
del c2